# "LeetCode 37 - Sudoku Solver: Backtracking"

- toc: false
- badges: true
- categories: [coding, python, leetcode, backtrack, DFS]
- comments: true

The LeetCode problem [Sudoku Solver](https://leetcode.com/problems/sudoku-solver/) is a classic backtracking problem.

According to [wikipedia](https://en.wikipedia.org/wiki/Backtracking):

> Backtracking incrementally builds candidates to the solutions, and abandons a candidate ("backtracks") as soon as it determines that the candidate cannot possibly be completed to a valid solution.

*`Cannot possibly be completed to a valid solution`* means there are some constraints that the candidate cannot satisfy. Therefore, those kind of problems are often called **constraint satisfaction problems**.

For example, in a Sudoku problem, we have following constraints that must be satisfied:

- Each of the digits 1-9 must occur exactly once in each row.

- Each of the digits 1-9 must occur exactly once in each column.

- Each of the digits 1-9 must occur exactly once in each of the 9 3x3 sub-boxes of the grid.

To check those three constraints, we construct three corresponding two-dimensional list as follows:

In [5]:
rows_available_list = [[True] * 9 for i in range(9)]
cols_available_list = [[True] * 9 for i in range(9)]
sub_boxes_available_list = [[True] * 9 for i in range(9)]

Each value in `rows_available_list[i]` indicates whether a number has been used in this row. For example, if `rows_available_list[i][j]` is `True`, that means you can still fill an empty cell in row `i+1` with number `j+1` (index starts at 0), but if it is `False`, that means the number `j+1` has already been used in this row. This is similar for `cols_available_list`.

For sub-boxes, the mapping relationship is a bit different. For cell `board[i][j]`, we check numbers' availability of this cell in `sub_boxes_available_list[i // 3 * 3 + j // 3]`.

So, every time we take a step towards the solution, we should check whether this step satisfies all constraints. Suppose in this step, we fill cell `board[i][j]` with number `num`, we need make sure this number is available in the row, colomn, and the sub-box that the cell belongs:

In [1]:
def check_availability(loc, num):
    i, j = loc
    index = num - 1
    return rows_available_list[i][index] and cols_available_list[j][index] and sub_boxes_available_list[i // 3 * 3 + j // 3][index]

Also, we need to write a function that updates those three constraint lists every time a step is taken towards the solution:

In [3]:
def update_constraint_list(loc, num, set):
    """
    loc: location of the cell
    set: True if the num is going to fill this cell, False if the num is to be removed from this cell
    """
    i, j = loc
    index = num - 1
    rows_available_list[i][index] = not set
    cols_available_list[j][index] = not set
    sub_boxes_available_list[i // 3 * 3 + j // 3][index] = not set

Of course, the constraint lists should have their initial values based on those already filled cells in the original board. By iterating through the original board, we can also figure out which cells are still empty.

In [6]:
board = [
    ["5", "3", ".", ".", "7", ".", ".", ".", "."], 
    ["6", ".", ".", "1", "9", "5", ".", ".", "."], 
    [".", "9", "8", ".", ".", ".", ".", "6", "."], 
    ["8", ".", ".", ".", "6", ".", ".", ".", "3"], 
    ["4", ".", ".", "8", ".", "3", ".", ".", "1"], 
    ["7", ".", ".", ".", "2", ".", ".", ".", "6"], 
    [".", "6", ".", ".", ".", ".", "2", "8", "."], 
    [".", ".", ".", "4", "1", "9", ".", ".", "5"], 
    [".", ".", ".", ".", "8", ".", ".", "7", "9"]
]

empty_cell_list = []

for i in range(9):
    for j in range(9):
        if board[i][j] != '.':
            update_constraint_list((i, j), int(board[i][j]), True)
        else:
            empty_cell_list.append((i, j))

Now, we are ready to write our solver with backtracking:

In [7]:
def solver(vacant_list, cur):
    if len(vacant_list) == cur:
        # we filled all empty cells, meaning we found a solution
        return True
    i, j = vacant_list[cur]
    for num in range(1, 10):
        if check_availability(vacant_list[cur], num):
            # take a step towards the solution
            board[i][j] = str(num)
            update_constraint_list((i, j), num, True)
            res = solver(vacant_list, cur + 1) # the next step
            if res:
                # we found a solution
                return True
            else:
                # we cannot find a solution after we take this step
                # backtracking ---> take a step backward
                board[i][j] = '.'
                update_constraint_list((i, j), num, False)
    return False

Give it a test and check the output:

In [8]:
solver(empty_cell_list, 0)
print(board)

[['5', '3', '4', '6', '7', '8', '9', '1', '2'], ['6', '7', '2', '1', '9', '5', '3', '4', '8'], ['1', '9', '8', '3', '4', '2', '5', '6', '7'], ['8', '5', '9', '7', '6', '1', '4', '2', '3'], ['4', '2', '6', '8', '5', '3', '7', '9', '1'], ['7', '1', '3', '9', '2', '4', '8', '5', '6'], ['9', '6', '1', '5', '3', '7', '2', '8', '4'], ['2', '8', '7', '4', '1', '9', '6', '3', '5'], ['3', '4', '5', '2', '8', '6', '1', '7', '9']]
